# Feature Engineering con *tsfresh* (HAR)

En este notebook se construirá un conjunto de *features* estadísticas masivas usando la librería **tsfresh** sobre el dataset HAR (ventanas de 2.56 s, 128 pasos, 9 canales).  

**Salidas principales:**  
- **Resumen de Características** (`../reports/tables/tsfresh_feature_summary.csv`): Archivo de metadatos que documenta la configuración usada, número de características iniciales, número tras la selección, y tiempos de cómputo.  
- **Set de Features Seleccionados** (`../artifacts/tsfresh_features_selected.npz`): Archivo comprimido que contiene la matriz final de *features* seleccionadas, junto con las etiquetas y metadatos.  
- **Figuras exploratorias** (`../reports/figures/`): Visualizaciones preliminares (PCA/UMAP) para evaluar la separabilidad de actividades en el espacio de *features*.  

## 1. Carga de datos y formato

* Reutilizar `har_processed.npz` (`X_train`, `y_train`, `channel_names`).
* Convertir cada muestra (ventana de 128 pasos, 9 canales) a un **DataFrame “long-form”** con columnas:

  * `id` = índice de muestra.
  * `time` = paso temporal (0–127).
  * `value` = valor de señal.
  * `kind` = canal (e.g., body_acc_x).
* Esto es el formato que *tsfresh* espera (`id`, `time`, `value`, `kind`).

In [7]:
# ============================================
# Paso 2 — Carga y formateo de datos (HAR → long-form tsfresh)
# ============================================

import os, gc
import numpy as np
import pandas as pd

# Configuración rápida
DATA_PATH = "../data/har_processed.npz"
CAP = 7000                 # pon un entero para trabajar sólo con las primeras N muestras
SAVE_LONG_PARQUET = False  # True si quieres cachear el long-form a disco (parquet)
PARQUET_PATH = "../artifacts/har_longform.parquet"

# 1) Cargar datos
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"No se encontró: {DATA_PATH}")

data = np.load(DATA_PATH, allow_pickle=True)
X_train = data["X_train"]          # (n, 128, 9)
y_train = data["y_train"]          # (n,)
channel_names = data.get("channel_names", None)

n, T, C = X_train.shape
print(f"[Carga] X_train: {X_train.shape} | y_train: {y_train.shape}")

# Asegurar nombres de canales
if channel_names is None or len(channel_names) != C:
    channel_names = np.array([f"ch{i}" for i in range(C)], dtype=object)
else:
    channel_names = channel_names.astype(str)

# CAP opcional (para pruebas rápidas)
if CAP is not None and CAP < n:
    X_train = X_train[:CAP]
    y_train = y_train[:CAP]
    n = X_train.shape[0]
    print(f"[CAP] Usando sólo {n} muestras.")

# 2) Construir long-form (id, time, kind, value)
def to_long_form(X, channel_names):
    """
    X: np.ndarray (n, T, C)
    channel_names: array-like de longitud C
    Devuelve: DataFrame con columnas ['id','time','kind','value']
    """
    n, T, C = X.shape
    dfs = []
    ids = np.repeat(np.arange(n, dtype=np.int32), T)       # (n*T,)
    times = np.tile(np.arange(T, dtype=np.int16), n)        # (n*T,)
    for c in range(C):
        vals = X[:, :, c].reshape(n * T)                    # (n*T,)
        df_c = pd.DataFrame({
            "id": ids,
            "time": times,
            "kind": pd.Categorical([channel_names[c]] * (n * T)),
            "value": vals.astype(np.float32, copy=False)
        })
        dfs.append(df_c)
        del vals, df_c
        if (c + 1) % 3 == 0:
            gc.collect()
    df_long = pd.concat(dfs, ignore_index=True)
    # Tipos compactos
    df_long["id"] = df_long["id"].astype("int32")
    df_long["time"] = df_long["time"].astype("int16")
    # 'kind' ya es categoría; 'value' es float32
    return df_long

df_long = to_long_form(X_train, channel_names)
print(f"[Long-form] df_long shape: {df_long.shape}  (~ n*T*C filas)")


# 3) Target alineado
target = pd.Series(y_train.astype(int, copy=False), name="target")
target.index.name = "id"
assert len(target) == n, "target y X no tienen el mismo número de muestras"

# 4) Sanity checks mínimos
assert {"id", "time", "kind", "value"}.issubset(df_long.columns), "Faltan columnas en long-form"
assert df_long["id"].min() == 0 and df_long["id"].max() == n - 1, "IDs fuera de rango"
assert df_long["time"].min() == 0 and df_long["time"].max() == T - 1, "Times fuera de rango"
assert df_long["kind"].nunique() == C, "Nº de canales (kind) no coincide con C"
assert np.isfinite(df_long["value"]).all(), "Hay NaN/Inf en 'value'"

print(f"[Checks] kinds={df_long['kind'].unique().tolist()[:5]}... (total {df_long['kind'].nunique()})")
print(f"[Target] shape={target.shape}, clases={sorted(map(int, pd.unique(target)))}")

# 5) Cachear long-form a parquet
if SAVE_LONG_PARQUET:
    os.makedirs(os.path.dirname(PARQUET_PATH), exist_ok=True)
    df_long.to_parquet(PARQUET_PATH, index=False)
    print(f"[Cache] Guardado long-form en: {PARQUET_PATH}")

gc.collect()


[Carga] X_train: (7352, 128, 9) | y_train: (7352,)
[CAP] Usando sólo 7000 muestras.
[Long-form] df_long shape: (8064000, 4)  (~ n*T*C filas)
[Checks] kinds=[np.str_('body_acc_x'), np.str_('body_acc_y'), np.str_('body_acc_z'), np.str_('body_gyro_x'), np.str_('body_gyro_y')]... (total 9)
[Target] shape=(7000,), clases=[1, 2, 3, 4, 5, 6]


0


## 2. Extracción de features con *tsfresh*

* Usar funciones de extracción de `tsfresh.feature_extraction.extract_features`.
* Parámetros:

  * `column_id="id"`, `column_sort="time"`, `column_kind="kind"`, `column_value="value"`.
  * Usar configuración eficiente: `EfficientFCParameters()` (para empezar sin explotar RAM).
* Generar un DataFrame de features por muestra.

In [8]:
# ============================================
# Paso 3 — Extracción de features con tsfresh
# ============================================
import time, gc
from tsfresh.feature_extraction import extract_features, MinimalFCParameters

# Configuración
TSFRESH_PARAMS = MinimalFCParameters()   # set reducido y eficiente
N_JOBS = 4                                 # paralelismo (ajusta según tu CPU)
DISABLE_PROGRESS = False                   # True si no quieres barra de progreso

# Extracción
t0 = time.time()
X_tsfresh = extract_features(
    df_long,
    column_id="id",
    column_sort="time",
    column_kind="kind",
    column_value="value",
    default_fc_parameters=TSFRESH_PARAMS,
    n_jobs=N_JOBS,
    disable_progressbar=DISABLE_PROGRESS
)
extract_time = round(time.time() - t0, 2)

print(f"[tsfresh] Extracción terminada en {extract_time} seg")
print(f"[tsfresh] Shape inicial: {X_tsfresh.shape}")

# Limpieza y sanity check
# Asegurar que index de X_tsfresh coincide con target (id = fila)
X_tsfresh = X_tsfresh.sort_index()
target = target.sort_index()

assert all(X_tsfresh.index == target.index), "Desalineación entre features y target"

# Remover columnas con NaN/Inf si existen (pueden aparecer en features raros)
na_cols = X_tsfresh.columns[X_tsfresh.isna().any()].tolist()
if na_cols:
    print(f"[Warning] {len(na_cols)} columnas con NaN -> se eliminan")
    X_tsfresh = X_tsfresh.drop(columns=na_cols)

inf_cols = [c for c in X_tsfresh.columns if np.isinf(X_tsfresh[c]).any()]
if inf_cols:
    print(f"[Warning] {len(inf_cols)} columnas con Inf -> se eliminan")
    X_tsfresh = X_tsfresh.drop(columns=inf_cols)

# Reporte final
print(f"[tsfresh] Shape tras limpieza: {X_tsfresh.shape}")


Feature Extraction: 100%|██████████| 20/20 [00:32<00:00,  1.64s/it]


[tsfresh] Extracción terminada en 40.56 seg
[tsfresh] Shape inicial: (7000, 90)
[tsfresh] Shape tras limpieza: (7000, 90)



## 3. Selección de features

* Usar `tsfresh.select_features` con `y_train` para filtrar solo features relevantes.
* Argumentos:

  * `target=y_train`.
  * `fdr_level` (ej. 0.05).
* Esto reducirá miles de features a un subconjunto estadísticamente significativo.


## 4. Validaciones rápidas

* **Sanity checks**:

  * Revisar número de features antes y después de la selección.
  * Confirmar ausencia de NaN/Inf.
* **Separabilidad preliminar**: PCA/UMAP sobre features seleccionados para ver agrupamiento por actividad.
* **Balance vs redundancia**: contar cuántos features se quedan por canal.


## 5. Exportación para paper y trazabilidad

* Guardar:

  * CSV de resumen (`../reports/tables/tsfresh_feature_summary.csv`) con:

    * `n_features_raw`, `n_features_selected`, parámetros de extracción, tiempo de cómputo.
  * Artefactos:

    * `../artifacts/tsfresh_features_selected.npz` (X_tsfresh, y, meta).
  * (Opcional) Figura PCA/UMAP de features seleccionados.


## 6. Limitaciones y plan de combinación

* Reconocer limitaciones:

  * *tsfresh* es intensivo en tiempo/memoria.
  * Riesgo de sobreajuste si no se controla selección.
* Plan experimental:

  * Comparar **solo PyTS** vs **solo tsfresh** vs **PyTS+tsfresh** en Notebook 04 (modelado).
